In [1]:
import pandas as pd
import gurobipy as grb
import numpy as np

In [2]:
inputFile='data3.xlsx'
spring_data=pd.read_excel(inputFile,sheet_name='data3', index_col=21) # index = session
spring_data

,n,Course,Course.Prefix,Course.Suffix,Department,First.Begin.Time,First.Days,N.Session,First.End.Time,First.End.Time.1,...,Max.Units,Min.Units,Mode,Reg.Count,Seats,Session,Term,Title,duration,duration_10
Section,,,,,,,,,,,,,,,,,,,,,
14028,1,ACCT-370,ACCT,370,ACCT,14:00:00,MW,2,15:50:00,15:50:00,...,4.0,4.0,C,37,47,1,20171,External Financial Reporting Issues,1.833333,2.0
14026,2,ACCT-370,ACCT,370,ACCT,10:00:00,TH,2,11:50:00,11:50:00,...,4.0,4.0,C,46,45,1,20171,External Financial Reporting Issues,1.833333,2.0
14027,3,ACCT-370,ACCT,370,ACCT,12:00:00,MW,2,13:50:00,13:50:00,...,4.0,4.0,C,47,48,1,20171,External Financial Reporting Issues,1.833333,2.0
14040,4,ACCT-371,ACCT,371,ACCT,08:00:00,MW,2,09:50:00,09:50:00,...,4.0,4.0,C,25,30,1,20171,Introduction to Accounting Systems,1.833333,2.0
14042,5,ACCT-371,ACCT,371,ACCT,10:00:00,TH,2,11:50:00,11:50:00,...,4.0,4.0,C,32,32,1,20171,Introduction to Accounting Systems,1.833333,2.0
14041,6,ACCT-371,ACCT,371,ACCT,10:00:00,MW,2,11:50:00,11:50:00,...,4.0,4.0,C,33,32,1,20171,Introduction to Accounting Systems,1.833333,2.0
14043,7,ACCT-371,ACCT,371,ACCT,12:00:00,TH,2,13:50:00,13:50:00,...,4.0,4.0,C,38,39,1,20171,Introduction to Accounting Systems,1.833333,2.0
14052,8,ACCT-372,ACCT,372,ACCT,16:00:00,TH,2,17:50:00,17:50:00,...,2.0,2.0,C,27,36,442,20171,Internal Reporting Issues,1.833333,2.0
14050,9,ACCT-372,ACCT,372,ACCT,12:00:00,TH,2,13:50:00,13:50:00,...,2.0,2.0,C,35,37,442,20171,Internal Reporting Issues,1.833333,2.0


In [4]:
ExpectedStudent = pd.read_excel('ES_final.xlsx',sheet_name='ES',index_col = 0)
print(ExpectedStudent)
print(len(ExpectedStudent))

         0    1    2    3    4    5    6    7    8    9   ...   260  261  262  \
Section                                                   ...                   
14028     37   37   37   37   37   37   37   37   37   37 ...    37   37   37   
14026     46   46   46   46   46   46   46   46   46   46 ...    46   46   46   
14027     47   47   47   47   47   47   47   47   47   47 ...    47   47   47   
14040     25   25   25   25   25   25   25   25   25   25 ...    25   25   25   
14042     32   32   32   32   32   32   32   32   32   32 ...    32   32   32   
14041     33   33   33   33   33   33   33   33   33   33 ...    33   33   33   
14043     38   38   38   38   38   38   38   38   38   38 ...    38   38   38   
14052     27   27   27   27   27   27   27   27   27   27 ...    27   27   27   
14050     35   35   35   35   35   35   35   35   35   35 ...    35   35   35   
14051     37   37   37   37   37   37   37   37   37   37 ...    37   37   37   
14057     40   40   40   40 

In [5]:
# Creating Capacity DataFrame [room] (45 Rooms * 5 weedays = 225 rows)
classroom = pd.read_excel('Marshall_Room_Capacity_Chart.xlsx',sheet_name='Room').iloc[:,0:2]

room_days1 = []
room_days2 = []
size_days1 = []
size_days2 = []
for i in range(0,45):
        room_days1.append(classroom.loc[i,'Room']+'_M')
        room_days1.append(classroom.loc[i,'Room']+'_T')
        room_days1.append(classroom.loc[i,'Room']+'_W')
        room_days1.append(classroom.loc[i,'Room']+'_H')
        room_days2.append(classroom.loc[i,'Room']+'_MW')
        room_days2.append(classroom.loc[i,'Room']+'_TH')
        size_days1.append(classroom.loc[i,'Size'])
        size_days1.append(classroom.loc[i,'Size'])
        size_days1.append(classroom.loc[i,'Size'])
        size_days1.append(classroom.loc[i,'Size'])
        size_days2.append(classroom.loc[i,'Size'])
        size_days2.append(classroom.loc[i,'Size'])
        
room1 = pd.DataFrame({'room_days' : room_days1,'Size' : size_days1,})
room2 = pd.DataFrame({'room_days' : room_days2,'Size' : size_days2,})
room = room1.append(room2,ignore_index=True)

In [6]:
# Creating the index sets
import numpy as np
I=spring_data.index # session
J1=room1.loc[:,'room_days'] # classroom
J2=room2.loc[:,'room_days']
J=room.loc[:,'room_days']
C=pd.DataFrame(room)
C.set_index('room_days',inplace=True)
# expected students
S=spring_data.loc[:,'N.Session'] # how many sections per week
T=spring_data.loc[:,'duration']

In [7]:
ES = ExpectedStudent
ES.columns = [J]
#test
#ES.loc[14029,'JKP212_TH']
#C.loc['ACC 306B_M', 'Size']

In [8]:
# I1 and I2
I1 = []
I2 = []
for i in I:
    if spring_data.loc[i,'N.Session']==1:
        I1.append(i)
    else:
        I2.append(i)

In [9]:
len(I1) + len(I2) == len(I)

True

In [10]:
mod = grb.Model()

# Defining decision variables

x={}
for i in I:
    for j in J:
        x[i,j] = mod.addVar(lb=0,name='x[{0},{1}]'.format(i,j),vtype=grb.GRB.BINARY)


#a=mod.addVar(name="Time Limit 1 Session",vtype=grb.GRB.INTEGER)
#b=mod.addVar(name="Time Limit 2 Sessions",vtype=grb.GRB.INTEGER)
        
u={}
for i in I:
    for j in J:
        u[i,j] = (ES.loc[i,j])/(C.loc[j,'Size'])

# Defining objective funtion
mod.setObjective(sum(x[i,j]*u[i,j] for i in I for j in J), sense=grb.GRB.MAXIMIZE) # objective


In [13]:
a = 4
b = 10
# Defining constraints - TIME
for j in J1:
    mod.addConstr(sum(T[i]*x[i,j] for i in I1) <= a)
    mod.addConstr(sum(x[i,j] for i in I2) == 0)
for j in J2:
    mod.addConstr(sum(T[i]*x[i,j] for i in I2) <= b)
    mod.addConstr(sum(x[i,j] for i in I1) == 0)

#mod.addConstr(a + b <= 14)
#mod.addConstr(a >= 0)
#mod.addConstr(b >= 0)
#mod.addConstr(b % 3 == 0)
    
# Defining constraints - CAPACITY
for j in J:
    for i in I:
        mod.addConstr(ES.loc[i,j]*x[i,j] <=C.loc[j,'Size'])
        
for i in I:
    mod.addConstr(sum(x[i,j] for j in J) == 1)

# Defining constrains - SECTION NUMBERS
#for i in I:
    #mod.addConstr(sum(x[i,j] for j in J) == S[i])

TypeError: unsupported operand type(s) for -: 'bool' and 'NoneType'

In [13]:
mod.optimize()

Optimize a model with 131704 rows, 130680 columns and 392040 nonzeros
Variable types: 0 continuous, 130680 integer (130680 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e-03, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 319.8599055
Presolve removed 130950 rows and 91338 columns (presolve time = 6s) ...
Presolve removed 130950 rows and 91338 columns
Presolve time: 6.27s
Presolved: 754 rows, 39342 columns, 78684 nonzeros
Variable types: 0 continuous, 39342 integer (39342 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1187351e+03   2.496286e+03   0.000000e+00      6s
    4181    4.2704133e+02   0.000000e+00   0.000000e+00      7s

Root relaxation: objective 4.270413e+02, 4181 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

In [14]:
print(mod.ObjVal)

426.1897430856841


In [15]:
result=pd.DataFrame('',index=I,columns=J)
for i in I:
    for j in J:
        value = x[i,j].x
        result.loc[i,j] = value
result

room_days,ACC 306B_M,ACC 306B_T,ACC 306B_W,ACC 306B_H,ACC201_M,ACC201_T,ACC201_W,ACC201_H,ACC205_M,ACC205_T,...,JKP112_MW,JKP112_TH,JKP202_MW,JKP202_TH,JKP204_MW,JKP204_TH,JKP210_MW,JKP210_TH,JKP212_MW,JKP212_TH
Section,,,,,,,,,,,,,,,,,,,,,
14028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('BIG_OUTPUT.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
result.to_excel(writer, sheet_name='output')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
answer = pd.DataFrame('',index = I, columns =['Classroom_DOW'])
for i in I:
    classroom = (result == 1).idxmax(axis=1)[i]
    answer.loc[i,'Classroom_DOW'] = classroom
answer

,Classroom_DOW
Section,
14028,JFF LL101_TH
14026,JFF241_TH
14027,JFF416_TH
14040,JFF241_MW
14042,ACC 306B_M
14041,ACC 306B_M
14043,BRI5_TH
14052,ACC 306B_M
14050,JFF417_TH


In [20]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('FINAL_SCHEDULE.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
answer.to_excel(writer, sheet_name='Schedule')

# Close the Pandas Excel writer and output the Excel file.
writer.save()